<a href="https://colab.research.google.com/github/dhivyapm/random-forest/blob/master/Random_Forest_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd # to read the dataset and convert the dataset to DataFrame which is in table format
import numpy as np # numpy is for computing, here I'm using this package for randomly select the rows from the dataset
import random # to randomly select the datapoints from dataset
df = pd.read_csv('/content/drive/My Drive/satelite.csv')
from random import randrange

In [3]:
df1 = df.copy()
df1 = df.drop('Column37',axis=1)
features =['Column17','Column18','Column19','Column20']
df = df.sample(frac=1)# shuffle the dataset

In [4]:
train_size = int(np.floor(0.80 * len(df))) #train_size =5791
X_train = df[features][:train_size] # X_train takes the values of the columns from 1 to 36 from the dataset upto the train_size rows(5791 rows)
y_train = df['Column37'][:train_size].values #y_train takes the last column values which is label upto the train_size(5791 rows)
X_test = df[features][train_size:] # X_test takes the values of the columns from 1 to 36 starting from rows 5791 upto the dataset length which is 6435
y_test = df['Column37'][train_size:].values # y_test takes the last column values which is label starting from rows 5791 upto the dataset length which is 6435


In [5]:
#calculating the bootstrap that is taking sample from train set and left out samples in dataset will be taken as out-of-bag which is used to test the model performance
def n_bootstrap(X_train, y_train):
  bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True)) #it as the index of the sample selected from the training set
  oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices] # it as the index of the left over samples in the training set
  X_bootstrap = X_train.iloc[bootstrap_indices].values # here it takes the values of the specific index from the training
  y_bootstrap = y_train[bootstrap_indices] # it takes the array elements of the specific indices(labels)
  X_oob = X_train.iloc[oob_indices].values # it takes the values of the left over samples in the training set
  y_oob = y_train[oob_indices] # it takes the array elements from the left over samples in the training set(labels)
    #q.put([X_bootstrap,y_bootstrap,X_oob,y_oob])
  return X_bootstrap, y_bootstrap, X_oob, y_oob # returns all the values of bootstrap and oob

In [6]:
# calculate the Entropy(measure of disorder)
#To which group does this sample belongs to
#Entropy controls how a Decision Tree decides to split the data.
#where entropy takes in a probability of a class within a node 
def entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return - (p * np.log2(p) + (1 - p) * np.log2(1-p))
         # these formula provided in these https://medium.com/coinmonks/what-is-entropy-and-why-information-gain-is-matter-4e85d46d2f01
         # in that they clearly explained the concept of entropy and Information-Gain

In [7]:
#calculate the information gain to compare the entropy before and after split
#Information_gain takes in a list of the classes from the left and right child and returns the information gain of that particular split.
def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = entropy(p_parent)
   
    IG_l = entropy(p_left)

    IG_r = entropy(p_right)

    IG = IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r
    
    return IG#return the information gain of parent and childs

In [8]:
def find_split_point(X_bootstrap, y_bootstrap, max_features):
    feature_ls = list()
    num_features = len(X_bootstrap[0])
    
    while len(feature_ls) <= max_features:
        feature_idx = random.sample(range(num_features), 1)
        if feature_idx not in feature_ls:
            feature_ls.extend(feature_idx)
    
    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
        for split_point in X_bootstrap[:,feature_idx]:
            left_child = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child = {'X_bootstrap': [], 'y_bootstrap': []}
            
            # split children for continuous variables
            if type(split_point) in [int, float]:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value <= split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
            # split children for categoric variables
            else:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value == split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
            
            split_info_gain = information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
            if split_info_gain > best_info_gain:
                best_info_gain = split_info_gain
                left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
                right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
                node = {'information_gain': split_info_gain, 
                        'left_child': left_child, 
                        'right_child': right_child, 
                        'split_point': split_point,
                        'feature_idx': feature_idx}
                
    
    return node

In [9]:
#function to build the tree with the boostrap values
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features) #creates the node with parent,right and left-child node
# Once we have a root node we will use the split_node() which will recusively split each internal node until each branch reached terminal node
    split_node(root_node, max_features, min_samples_split, max_depth, 1) 
    return root_node

In [10]:
def split_node(node, max_features, min_samples_split, max_depth, depth):
    left_child = node['left_child']
    right_child = node['right_child']    

    del(node['left_child'])
    del(node['right_child'])
    
    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return
    
    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node
    
    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = node['right_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
        split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1)
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = node['left_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

In [11]:
#function for deciding the terminal-node (leaf-node)
def terminal_node(node):
    y_bootstrap = node['y_bootstrap']
    pred = max(y_bootstrap, key = y_bootstrap.count)
    return pred

In [12]:
def oob_score(tree, X_test, y_test):
    mis_label = 0
    for i in range(len(X_test)):
        pred = predict_tree(tree, X_test[i])
        if pred != y_test[i]:
            mis_label += 1
    return mis_label / len(X_test)

In [13]:
def predict_tree(tree, X_test):
    feature_idx = tree['feature_idx']
    
    if X_test[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        else:
            return tree['right_split']

In [21]:
def random_forest(X_train, y_train, X_test, n_estimators, max_features, max_depth, min_samples_split):
  tree_ls = list()
  oob_ls = list()
  for i in range(n_estimators):
    X_bootstrap, y_bootstrap, X_oob, y_oob = n_bootstrap(X_train, y_train)
    tree = build_tree(X_bootstrap, y_bootstrap, max_features, max_depth, min_samples_split)
    tree_ls.append(tree)
    oob_error = oob_score(tree, X_oob, y_oob)
    oob_ls.append(oob_error)
  print("OOB estimate: {:.2f}".format(np.mean(oob_ls)))
  predictions = predict_rf(tree_ls, X_test)
  return predictions

In [16]:
def predict_rf(tree_ls, X_test):
    pred_ls = list()
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test.values[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key = ensemble_preds.count)
        pred_ls.append(final_pred)
    return np.array(pred_ls)

In [17]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [24]:
import time
start = time.time()
preds = random_forest(X_train, y_train, X_test, n_estimators=1, max_features=2, max_depth=10, min_samples_split=2)
end = time.time()
print('Time taken in mins:',(end-start)/60,'mins')

OOB estimate: 0.82
Time taken in mins: 2.4481837232907613 mins


In [ ]:
scores =list()
accuracy = accuracy_metric(y_test, preds)
scores.append(accuracy)
print('scores:',scores)

scores: [30.691530691530694]


In [25]:
import time
start = time.time()
preds = random_forest(X_train, y_train, X_test, n_estimators=2, max_features=6, max_depth=10, min_samples_split=2)
end = time.time()
print('Time taken in mins:',(end-start)/60,'mins')

OOB estimate: 0.70
Time taken in mins: 12.205070380369822 mins


In [ ]:
scores =list()
accuracy = accuracy_metric(y_test, preds)
scores.append(accuracy)
print('scores:',scores)

scores: [30.76923076923077]
